In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
n_ts = [10_000, 100_000, 1_000_000, 5_000_000, 10_000_000]
times = [pd.read_csv(f'../results/time_{size}.csv').drop(labels='Unnamed: 0', axis=1) for size in n_ts]
times = pd.concat(times)
times = times[['length', 'time', 'cpus']]
times['Cost (Dollars)'] = times['time'] * 26.688 / 60

In [ ]:
evaluation = []
for size in n_ts:
    fcast = pd.read_parquet(f'../results/forecasts_{size}.parquet')
    models = fcast.drop(labels=['ds', 'cutoff', 'y'], axis=1).columns
    eval_models = {}
    for model in models:
        mse = (fcast['y'] - fcast[model]) ** 2
        eval_models[model] = np.mean(mse)
    eval_models['length'] = size
    eval_models['n_windows'] = np.unique(fcast.groupby(['unique_id', 'cutoff']).size())
    evaluation.append(pd.DataFrame(eval_models, index=[0]))
evaluation = pd.concat(evaluation).reset_index(drop=True)

In [ ]:
evaluation = evaluation.rename(columns = {
    'croston_classic': 'Croston',
    'seasonal_naive_season_length-7': 'SeasNaive',
    'naive': 'Naive',
    'adida': 'ADIDA',
    'historic_average': 'HistoricAverage',
    'seasonal_window_average_season_length-7_window_size-4': 'SeasWindowAverage',
    'imapa': 'iMAPA',
    'window_average_window_size-7': 'WindowAverage',
    'seasonal_exponential_smoothing_season_length-7_alpha-0.9': 'SeasExpSmooth',
    'n_windows': 'CVWindows'
})

In [ ]:
results = times.merge(evaluation, how='left', on=['length'])

In [ ]:
results = results.rename(columns={
    'length': 'N time series',
    'time': 'Time (mins)',
    'cpus': 'N cpus'

})

In [ ]:
model_cols = [
    'Croston', 'SeasNaive', 'Naive', 'ADIDA',
    'HistoricAverage', 'SeasWindowAverage', 'iMAPA', 'WindowAverage',
    'SeasExpSmooth'
]
extra_cols = [
    'N time series', 'Time (mins)', 
    'N cpus', 'CVWindows', 'Cost (Dollars)'
]

In [ ]:
results[model_cols] = results[model_cols].applymap('{:.4f}'.format)

In [ ]:
results[['Time (mins)', 'CVWindows']] = results[['Time (mins)', 'CVWindows']].applymap('{:.2f}'.format)

In [ ]:
results['N time series'] = results['N time series'].apply('{:,}'.format) 

In [ ]:
results['Cost (Dollars)'] = results['Cost (Dollars)'].apply('${:.2f}'.format)

In [ ]:
results = results[extra_cols + model_cols]

In [ ]:
print(results[extra_cols].to_markdown(index=False))

In [ ]:
print(results[['N time series'] + model_cols].to_markdown(index=False))